source API URL : "https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-01-01&end_date=2024-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum"

JSON Target File Path : "abfss://bronze@datalakestorageaccountname.dfs.core.windows.net/bronze/weather-data/
"

In [0]:
weatherDataSourceAPIBaseURL = "https://archive-api.open-meteo.com/v1/archive?latitude="
weatherDataSourceAPIURLOptions = "&daily=temperature_2m_max,temperature_2m_min,rain_sum"

weatherDataSinkLayerName = 'bronze'
weatherDataSinkStorageAccountName = 'adlsalidatalakehousedev'
weatherDataSinkFolderName = 'weather-data'

weatherDataSinkFolderPath = f"abfss://{weatherDataSinkLayerName}@{weatherDataSinkStorageAccountName}.dfs.core.windows.net/{weatherDataSinkFolderName}"

In [0]:
import requests
import json
import pandas as pds

In [0]:
geoLocationsDF = spark.sql("SELECT latitude, longitude, marketName FROM pricing_analytics.silver.geo_location_silver")

In [0]:
weatherDataAPIResponseList = []
for geoLocations in geoLocationsDF.collect():
    weatherDataSourceAPIURL = f"{weatherDataSourceAPIBaseURL}{geoLocations['latitude']}&longitude={geoLocations['longitude']}&start_date=2023-01-01&end_date=2023-12-31{weatherDataSourceAPIURLOptions}"
    weatherDataAPIResponse = requests.get(weatherDataSourceAPIURL).json()
    print(weatherDataAPIResponse)
    weatherDataAPIResponse["marketName"] = geoLocations["marketName"]
    weatherDataAPIResponseJson = json.dumps(weatherDataAPIResponse)
    if isinstance(weatherDataAPIResponse, dict):
        weatherDataAPIResponseList.append(weatherDataAPIResponseJson)

weatherDataPandasDF = pds.DataFrame(weatherDataAPIResponseList)

weatherDataSparkDF = spark.createDataFrame(weatherDataPandasDF)

(weatherDataSparkDF
 .write
 .mode("overwrite")
 .json(weatherDataSinkFolderPath)

)


    
